In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:

#-----------------one hot encoding of labels----------------------
def label_encoder(y):
    one_hot_y=[]
    y=y.T
    for d in y:
        n=np.zeros(10)
        n[d]=1
        one_hot_y.append(n)
    return np.array(one_hot_y).T
    

In [3]:
class sigmoid_layer():
    def __init__(self, n):
        self.n=n
        self.weight=np.random.randn(self.n,x.shape[0])
        self.bias=np.random.randn(self.n,1)
        
    def der():
        pass
        
    def forward(self, x):
        self.z= np.dot(self.weight,x)+self.bias
        self.a=1/(1+np.exp(x))
        return self.a
    
        
    def backward(self, delta_w):
        delta=np.dot(delta_w,self.der(self.z))   
        dw=np.dot(delta, self.x)
        db=delta
        self.weight-=0.1*dw
        self.bias-=0.1*db
        return np.dot(delta,self.weight)


In [4]:
class loss():
    def __init__(self):
        pass

    def mse_loss_function(y_true, y_pred):
        n = y_pred.shape[1]
        cost = (1./(2*n)) * np.sum((y_true - y_pred) ** 2)
        return cost

    def mse_loss_derivative(y_true, y_pred):
        cost_prime = y_pred - y_true
        return cost_prime

    def cross_entropy_loss(y_pred,y_true):
        y_true=label_encoder(y_true)
        loss=[]
        for i in range(y_true.shape[1]):
            loss.append(-np.dot(y_true[:,i],np.log(1e-15+y_pred[:,i])))
        return loss

    def cross_entropy_derivative(y_pred, y_true):    
        y_true=label_encoder(y_true)
        return (y_pred-y_true)/y_pred.shape[1]



In [5]:
class softmax_layer():
    def __init__(self,n):
        self.lr=0.01
        self.weight=np.random.randn(n[0],n[1])* np.sqrt(1 / n[1])
        self.bias=np.random.rand(n[0],1)
        self.old_w=self.weight
    
    def forward(self,x):
        self.old_w=self.weight
        self.x=x
        self.z= np.dot(self.weight,self.x)+self.bias 
        arr=[]
        for i in range(self.x.shape[1]):
            exps=np.exp(self.z[:,i]-np.max(self.z[:,i]))
            sfx=exps/np.sum(exps)
            arr.append(sfx)
        self.a=np.array(arr).T
        return self.a

    
    def backward(self, delta_w):
        delta=delta_w  
        dw=np.dot(delta, self.x.T)
        db=np.expand_dims(delta.mean(axis=1),1)
        self.weight-=self.lr*dw
        self.bias-=self.lr*db
        return np.dot( self.old_w.T, delta)

    

In [6]:
class relu_layer:
    def __init__(self, n):
        
        self.lr=0.01
        self.weight=np.random.randn(n[0],n[1])* np.sqrt(1 / n[1])
        self.bias=np.random.rand(n[0],1)
        self.old_w=self.weight
        
    def der(self,x):
        x[x>0]=1
        x[x<=0]=0
        return x
        
        
    def forward(self, x):
        self.old_w=self.weight
        self.x=x
        self.z= np.dot(self.weight,self.x)+self.bias
        self.a=np.maximum(0,self.z)
        return self.a
    
    def backward(self, delta_w):
        delta=delta_w*self.der(self.z) 
        dw=np.dot(delta, self.x.T)
        db=np.expand_dims(delta.mean(axis=1),1)
        self.weight-=self.lr*dw
        self.bias-=self.lr*db
        return np.dot(self.old_w.T, delta)


In [7]:
class neural_network:
    def __init__(self):
        self.r1=relu_layer([300, 784])
        self.r2=relu_layer([100,300])
        self.r3=relu_layer([200,100])
        self.s1=softmax_layer([10,200])
        
    def forward(self, a0): 
        a1=self.r1.forward(a0)
        a2=self.r2.forward(a1)
        a3=self.r3.forward(a2)
        out=self.s1.forward(a3)
        return out
    
    def backward(self,y_pred,y_true):
        delta_L=loss.cross_entropy_derivative(y_pred,y_true)
        delta_4=self.s1.backward(delta_L)
        delta_3=self.r3.backward(delta_4)
        delta_2=self.r2.backward(delta_3)
        delta_1=self.r1.backward(delta_2)
  
        
    def train(self, x, y, batch_size, epochs):
        for e in range(epochs):
            if x.shape[1]%batch_size==0:
                n_batches=x.shape[1]//batch_size
            else:
                n_batches=x.shape[1]//batch_size-1
            print("n_batches", n_batches)
                
            batches_x=[x[:,batch_size*i:batch_size*(i+1)] for i in range(n_batches)]
            batches_y=[y[:,batch_size*i:batch_size*(i+1)] for i in range(n_batches)]
            
            
            for bx,by in zip(batches_x, batches_y):
                out=self.forward(bx)
                self.backward(out,by)
                ans=[]
                for i in range(out.shape[1]):
                    ans.append(np.argmax(out[:,i]))
                l=loss.cross_entropy_loss(out,by)
                a=accuracy_score(ans,by.squeeze(axis=0))
                print("epoch: {}, loss: {}, acc: {}".format(e,np.mean(l), a))


    

In [10]:
nn=neural_network()
nn.train(x_train,y_train, 2000, 50)

n_batches 24
epoch: 0, loss: 2.8210179121058565, acc: 0.1035
epoch: 0, loss: 2.602667249316266, acc: 0.1005
epoch: 0, loss: 2.45042050016387, acc: 0.107
epoch: 0, loss: 2.3738348242214, acc: 0.109
epoch: 0, loss: 2.331977534380676, acc: 0.121
epoch: 0, loss: 2.3057588112306737, acc: 0.162
epoch: 0, loss: 2.291570095704979, acc: 0.1635
epoch: 0, loss: 2.260819690872537, acc: 0.191
epoch: 0, loss: 2.2555882024662615, acc: 0.2005
epoch: 0, loss: 2.232253020982335, acc: 0.245
epoch: 0, loss: 2.229654682907394, acc: 0.275
epoch: 0, loss: 2.2194485014486824, acc: 0.288
epoch: 0, loss: 2.214072453612866, acc: 0.2755
epoch: 0, loss: 2.2079664288902374, acc: 0.303
epoch: 0, loss: 2.2051078194676377, acc: 0.2735
epoch: 0, loss: 2.184355282319266, acc: 0.32
epoch: 0, loss: 2.1732236285635955, acc: 0.324
epoch: 0, loss: 2.177528943404593, acc: 0.309
epoch: 0, loss: 2.160744621241282, acc: 0.3515
epoch: 0, loss: 2.1620103836389206, acc: 0.3455
epoch: 0, loss: 2.152247707218483, acc: 0.3645
epoch: 0

epoch: 7, loss: 1.080018087046367, acc: 0.7725
epoch: 7, loss: 1.0993428181170593, acc: 0.74
epoch: 7, loss: 1.0585372736845953, acc: 0.772
epoch: 7, loss: 1.0765449483328786, acc: 0.7725
epoch: 7, loss: 1.0550589503779857, acc: 0.766
epoch: 7, loss: 1.0564096761401016, acc: 0.767
epoch: 7, loss: 1.0136238756188245, acc: 0.7895
epoch: 7, loss: 1.057075672778214, acc: 0.76
epoch: 7, loss: 1.038001165122657, acc: 0.7785
epoch: 7, loss: 1.060515449598771, acc: 0.7625
epoch: 7, loss: 1.03817966818053, acc: 0.7645
epoch: 7, loss: 1.0054162014752603, acc: 0.7825
epoch: 7, loss: 1.0120713357790658, acc: 0.783
epoch: 7, loss: 0.9991727898392814, acc: 0.779
epoch: 7, loss: 1.009700775449356, acc: 0.7745
epoch: 7, loss: 1.0204072365946395, acc: 0.773
epoch: 7, loss: 1.0155160206331864, acc: 0.759
epoch: 7, loss: 1.0210220152722778, acc: 0.7755
epoch: 7, loss: 1.0002928261257902, acc: 0.787
n_batches 24
epoch: 8, loss: 0.9877566051170847, acc: 0.7675
epoch: 8, loss: 0.9962698110191561, acc: 0.776

epoch: 14, loss: 0.6186983019084359, acc: 0.849
epoch: 14, loss: 0.6577039247593232, acc: 0.829
epoch: 14, loss: 0.637472324929161, acc: 0.8265
epoch: 14, loss: 0.6387239337765952, acc: 0.8385
epoch: 14, loss: 0.5911534009619731, acc: 0.856
epoch: 14, loss: 0.6413112127950558, acc: 0.8295
epoch: 14, loss: 0.6198319383867239, acc: 0.8445
epoch: 14, loss: 0.6394218406443274, acc: 0.8385
epoch: 14, loss: 0.6461095776465069, acc: 0.8365
epoch: 14, loss: 0.6021470935299693, acc: 0.846
epoch: 14, loss: 0.6121443033940606, acc: 0.851
epoch: 14, loss: 0.6053262076249826, acc: 0.844
epoch: 14, loss: 0.6090792213344082, acc: 0.84
epoch: 14, loss: 0.6338517063885369, acc: 0.843
epoch: 14, loss: 0.623210534514651, acc: 0.834
epoch: 14, loss: 0.6235620712625194, acc: 0.843
epoch: 14, loss: 0.612799610910495, acc: 0.8445
n_batches 24
epoch: 15, loss: 0.6195248015512456, acc: 0.8345
epoch: 15, loss: 0.618705310911914, acc: 0.841
epoch: 15, loss: 0.6058797932516933, acc: 0.8465
epoch: 15, loss: 0.6003

epoch: 21, loss: 0.4740829440399786, acc: 0.872
epoch: 21, loss: 0.5235416380783438, acc: 0.851
epoch: 21, loss: 0.5041845528415357, acc: 0.857
epoch: 21, loss: 0.506772972770014, acc: 0.8635
epoch: 21, loss: 0.4536423432121635, acc: 0.881
epoch: 21, loss: 0.4980322484823531, acc: 0.8645
epoch: 21, loss: 0.482636468278811, acc: 0.87
epoch: 21, loss: 0.5004948980322793, acc: 0.859
epoch: 21, loss: 0.5113610754262815, acc: 0.8585
epoch: 21, loss: 0.466890024802701, acc: 0.8745
epoch: 21, loss: 0.47920984310684467, acc: 0.871
epoch: 21, loss: 0.4717666266299756, acc: 0.8745
epoch: 21, loss: 0.4745345760642045, acc: 0.8705
epoch: 21, loss: 0.5057216316404679, acc: 0.8695
epoch: 21, loss: 0.4891737234646627, acc: 0.862
epoch: 21, loss: 0.489312928204205, acc: 0.869
epoch: 21, loss: 0.47970361465557687, acc: 0.8725
n_batches 24
epoch: 22, loss: 0.4918812882499841, acc: 0.864
epoch: 22, loss: 0.4906939676542045, acc: 0.8665
epoch: 22, loss: 0.47523659336679175, acc: 0.8755
epoch: 22, loss: 0.

epoch: 28, loss: 0.4261046031270131, acc: 0.879
epoch: 28, loss: 0.44482576186371714, acc: 0.872
epoch: 28, loss: 0.40435474107540875, acc: 0.887
epoch: 28, loss: 0.45784899828054515, acc: 0.871
epoch: 28, loss: 0.440758352568652, acc: 0.877
epoch: 28, loss: 0.4437468617396269, acc: 0.8805
epoch: 28, loss: 0.38765666612966343, acc: 0.8945
epoch: 28, loss: 0.42572913558537673, acc: 0.8855
epoch: 28, loss: 0.4162719237305083, acc: 0.887
epoch: 28, loss: 0.43371760887161825, acc: 0.874
epoch: 28, loss: 0.44369312509469283, acc: 0.874
epoch: 28, loss: 0.4010518183245596, acc: 0.8875
epoch: 28, loss: 0.4128799040823506, acc: 0.8875
epoch: 28, loss: 0.40517137291182875, acc: 0.889
epoch: 28, loss: 0.4085022072233132, acc: 0.882
epoch: 28, loss: 0.4430540808357978, acc: 0.881
epoch: 28, loss: 0.4240296696321127, acc: 0.881
epoch: 28, loss: 0.4249127381568111, acc: 0.8825
epoch: 28, loss: 0.4137740433308694, acc: 0.8835
n_batches 24
epoch: 29, loss: 0.4268817758376639, acc: 0.879
epoch: 29, lo

epoch: 35, loss: 0.386553295713692, acc: 0.8915
epoch: 35, loss: 0.40319854233063257, acc: 0.884
epoch: 35, loss: 0.3637644322501847, acc: 0.8995
epoch: 35, loss: 0.41888419210717853, acc: 0.8795
epoch: 35, loss: 0.40434330477272706, acc: 0.885
epoch: 35, loss: 0.4063301118096119, acc: 0.8905
epoch: 35, loss: 0.3490907337078111, acc: 0.9015
epoch: 35, loss: 0.3826829173353445, acc: 0.8945
epoch: 35, loss: 0.3779737726181498, acc: 0.896
epoch: 35, loss: 0.3944603712803589, acc: 0.8855
epoch: 35, loss: 0.4033246628454809, acc: 0.8855
epoch: 35, loss: 0.36300818291860154, acc: 0.8935
epoch: 35, loss: 0.3737181084798081, acc: 0.9005
epoch: 35, loss: 0.3652835772656965, acc: 0.8965
epoch: 35, loss: 0.36972998582943406, acc: 0.891
epoch: 35, loss: 0.4059730531505791, acc: 0.8855
epoch: 35, loss: 0.3866708006711367, acc: 0.8915
epoch: 35, loss: 0.387661611179124, acc: 0.8875
epoch: 35, loss: 0.37539915142920727, acc: 0.8935
n_batches 24
epoch: 36, loss: 0.38776986836809535, acc: 0.8935
epoch:

epoch: 42, loss: 0.35323662604668393, acc: 0.9135
epoch: 42, loss: 0.3414020390675061, acc: 0.9055
epoch: 42, loss: 0.3605709849233976, acc: 0.8975
epoch: 42, loss: 0.3760304460427562, acc: 0.8955
epoch: 42, loss: 0.3372421820879764, acc: 0.905
epoch: 42, loss: 0.39313567865152377, acc: 0.889
epoch: 42, loss: 0.3807095909153536, acc: 0.8915
epoch: 42, loss: 0.3811763792200119, acc: 0.8965
epoch: 42, loss: 0.32385367800499093, acc: 0.909
epoch: 42, loss: 0.3544818397090421, acc: 0.906
epoch: 42, loss: 0.353323890250088, acc: 0.902
epoch: 42, loss: 0.36859481613463924, acc: 0.8905
epoch: 42, loss: 0.3765236232683459, acc: 0.8935
epoch: 42, loss: 0.33847373950664167, acc: 0.901
epoch: 42, loss: 0.3482595028934388, acc: 0.908
epoch: 42, loss: 0.3385596089930576, acc: 0.9045
epoch: 42, loss: 0.3441949470761413, acc: 0.9
epoch: 42, loss: 0.3811975248597654, acc: 0.8885
epoch: 42, loss: 0.3627156216216927, acc: 0.9
epoch: 42, loss: 0.36345236579043877, acc: 0.8965
epoch: 42, loss: 0.350505082

epoch: 49, loss: 0.35057290719949336, acc: 0.8995
epoch: 49, loss: 0.3293159346409898, acc: 0.909
epoch: 49, loss: 0.33508791933704346, acc: 0.9155
epoch: 49, loss: 0.3231570092225782, acc: 0.9085
epoch: 49, loss: 0.34191915943884144, acc: 0.902
epoch: 49, loss: 0.3566544431500346, acc: 0.9005
epoch: 49, loss: 0.31831121144856606, acc: 0.912
epoch: 49, loss: 0.37463702030049306, acc: 0.896
epoch: 49, loss: 0.3637125024715848, acc: 0.894
epoch: 49, loss: 0.36270217183066683, acc: 0.9
epoch: 49, loss: 0.30577986093721743, acc: 0.914
epoch: 49, loss: 0.3344879548399315, acc: 0.9095
epoch: 49, loss: 0.3358788884707834, acc: 0.9045
epoch: 49, loss: 0.34993697398896506, acc: 0.8975
epoch: 49, loss: 0.35712932957773275, acc: 0.9
epoch: 49, loss: 0.3212304063576119, acc: 0.906
epoch: 49, loss: 0.33014336852832127, acc: 0.911
epoch: 49, loss: 0.3190545083072242, acc: 0.9105
epoch: 49, loss: 0.3257435267329304, acc: 0.902
epoch: 49, loss: 0.36309977549636263, acc: 0.8965
epoch: 49, loss: 0.34572

In [8]:
df=pd.read_csv("mnsit/nnn_mnist_scratch/mnist/mnist_train.csv")
x=df.iloc[:,1:]
y=df.iloc[:,0]

x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=12)
x_train=x_train.to_numpy()
x_val=x_val.to_numpy()
y_train=np.expand_dims(y_train.to_numpy(),1)
y_val=np.expand_dims(y_val.to_numpy(),1)

In [9]:
x_train=x_train.astype('float32')
x_val=x_val.astype('float32')
x_train/=255
x_val/=255

x_train,x_val, y_train, y_val=x_train.T,x_val.T, y_train.T, y_val.T


In [11]:
ans=nn.forward(x_val)

a=[]
for i in range(ans.shape[1]):
    a.append(np.argmax(ans[:,i]))
    
accuracy_score(a,y_val.squeeze(axis=0))

0.9040833333333333